<a href="https://colab.research.google.com/github/25maikelo/ProyectoTopicosI/blob/main/proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import panel as pn
import panel.widgets as pnw
pn.extension('plotly')

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import math
from scipy.stats import wrapcauchy, levy_stable
import plotly.figure_factory as ff

In [33]:
# Cargar dataset
dataset = pd.read_csv('mxmh_survey_results.csv')

# Limpiar dataset
df = dataset.dropna()

# Convertir la columna 'Age' a enteros
df['Age'] = df['Age'].astype(int)

# Convertir las columnas que comienzan con 'Frequency' a valores numéricos
frequency_mapping = {'Never': 0, 'Rarely': 1, 'Sometimes': 2, 'Very frequently': 3}
frequency_columns = [col for col in df.columns if 'Frequency' in col]

df[frequency_columns] = df[frequency_columns].replace(frequency_mapping)


In [115]:
# Functions
def age_range_bar_chart(dataframe, title):
    # Definir los rangos de edad
    bins = [0, 10, 20, 30, 40, 50, 60, 70, 80]

    # Crear una nueva columna 'Age Range' utilizando pd.cut
    dataframe['Age Range'] = pd.cut(dataframe['Age'], bins=bins, labels=[f'{i}-{i+9}' for i in range(10, 90, 10)])

    # Calcular el promedio de 'Hours per day' para cada rango de edad
    average_hours_per_day = dataframe.groupby('Age Range')['Hours per day'].mean().reset_index()

    # Crear el gráfico de barras con plotly.graph_objects
    fig = go.Figure(data=[go.Bar(x=average_hours_per_day['Age Range'], y=average_hours_per_day['Hours per day'])])

    # Configurar el diseño del gráfico
    fig.update_layout(
        title=title,
        xaxis=dict(title='Rango de Edad'),
        yaxis=dict(title='Promedio de Horas por Día')
    )

    return fig

def generate_bar_plot(x, title):
  trace = go.Histogram(
      x=x,
  )
  layout = go.Layout(
      title=title,
      scene=dict(
          xaxis=dict(title=title),
          yaxis=dict(title='Frecuencia')
      )
  )
  fig = go.Figure(data=[trace], layout=layout)
  fig.update_layout(width=400, height=300)

  return fig

def generate_disp_plot(x, y, title):
  trace = go.Scattergl(
      x=x,
      y=y,
      mode='markers'  # Indicamos que queremos puntos en lugar de líneas
  )
  layout = go.Layout(
      title=f'Diagrama de Dispersión para {title}',
      xaxis=dict(title='X'),
      yaxis=dict(title='Y')
  )
  fig = go.Figure(data=[trace], layout=layout)
  fig.update_layout(width=400, height=300)

  return fig

def generate_heatmap(dataframe, title):

  # Seleccionar solo las columnas numéricas
  numeric_columns = dataframe.select_dtypes(include=['number']).columns
  numeric_dataframe = dataframe[numeric_columns]

  correlation_matrix = dataframe.corr()

  fig = ff.create_annotated_heatmap(
      z=correlation_matrix.values,
      x=correlation_matrix.columns.tolist(),
      y=correlation_matrix.columns.tolist(),
      colorscale='Viridis'
  )

  fig.update_layout(
      title=title,
      xaxis=dict(title='Columnas'),
      yaxis=dict(title='Columnas')
  )

  return fig

def get_pie(dataframe, title):
  fig = go.Figure(data=[go.Pie(labels=dataframe.index, values=dataframe.values)])
  fig.update_layout(title=title)
  return fig

In [119]:
# DAHSBOARD COMPLETO
# TODO: Acomodar paneles visuales
# TODO: Que el gráfico de pastel se acomode a los rangos de edad

# Convertir la columna 'Primary streaming service' a una lista
platform_options = df['Primary streaming service'].unique().tolist()
genre_options = df['Fav genre'].unique().tolist()

min_age_slider = pnw.IntSlider(name='Edad mínima', start=df['Age'].min(), end=df['Age'].max(), value=df['Age'].min())
max_age_slider = pnw.IntSlider(name='Edad máxima', start=df['Age'].min(), end=df['Age'].max(), value=df['Age'].max())
platform_multi_select = pnw.MultiSelect(name='Plataformas', options=platform_options, value=platform_options)
frequency_select = pnw.Select(name='Frecuencia', options=df.filter(like='Frequency').columns.tolist(), value=df.filter(like='Frequency').columns[0])

# Función para aplicar los filtros y actualizar el DataFrame
def apply_filters(min_age, max_age, platforms, frequency_column):
    filtered_df = df[(df['Age'] >= min_age_slider.value) & (df['Age'] <= max_age_slider.value) &
                     (df['Primary streaming service'].isin(platform_multi_select.value))]

    # Obtener las columnas numéricas excluyendo aquellas que contienen "Frequency" (excepto la especificada)
    numeric_columns = [col for col in filtered_df.select_dtypes(include='number').columns
                       if 'Frequency' not in col or col == frequency_column]

    # Filtrar el DataFrame para incluir solo las columnas numéricas deseadas
    filtered_df = filtered_df[numeric_columns]

    return filtered_df


@pn.depends(min_age_slider.param.value, max_age_slider.param.value, platform_multi_select.param.value, frequency_select.param.value)
def dashboard(min_age, max_age, platforms, frequency_column):

    # Capa 1 - Filtrar dataset
    filtered_df = apply_filters(min_age, max_age, platforms, frequency_column)

    # Capa 2 - Representación del nuevo dataset
    # Grafico de barras de edad x tiempo de escucha general
    age_time_bar = age_range_bar_chart(dataframe=filtered_df, title='Tiempo de escucha por edad')

    # Gráfico de pastel para el género más escuchado
    genre_counts = df['Fav genre'].value_counts()
    genre_pie = get_pie(genre_counts, title="Género más escuchado")

    # Capa 3 - Métricas
    # Correlacional: Frecuencia de X género musical contra Y padecimiento

    genre_corr_heatmap = generate_heatmap(df[[frequency_column, 'Anxiety', 'Depression', 'Insomnia', 'OCD']],
                                          "Correlación entre Frecuencia de Géneros y Padecimientos")

    # Correlacional: Tiempo de escucha total x padecimiento
    time_conditions = ['Hours per day', 'Anxiety', 'Depression', 'Insomnia', 'OCD']
    time_corr_heatmap = generate_heatmap(filtered_df[time_conditions], "Correlación entre Tiempo de Escucha y Padecimientos")

    return pn.Row(pn.Column(age_time_bar, genre_pie), pn.Column(genre_corr_heatmap, time_corr_heatmap))

# Mostrar Dashboard
pn.Column(min_age_slider, max_age_slider, platform_multi_select, frequency_select, dashboard).servable()

Column(sizing_mode='stretch_width')
    [0] IntSlider(end=89, name='Edad mínima', sizing_mode='stretch_width', start=10, value=10)
    [1] IntSlider(end=89, name='Edad máxima', sizing_mode='stretch_width', start=10, value=89)
    [2] MultiSelect(name='Plataformas', options=['Spotify', 'YouTube Music...], sizing_mode='stretch_width', value=['Spotify', 'YouTube Music...])
    [3] Select(name='Frecuencia', options=['Frequency [Classical]', ...], sizing_mode='stretch_width', value='Frequency [Classical]')
    [4] ParamFunction(function, _pane=Row, defer_load=False, sizing_mode='stretch_width')

In [7]:
#Selectores
datasets = pnw.RadioButtonGroup(name='Data', options=['Origen', 'Personalizada'], margin=(0,0,0,0), width=200)

#NO TOMADAS EN CUENTA
#BPM
#OCD	[1,10]


#ENTEROS
#Age	[10,89]

#DECIMALES
#Anxiety	[1,10]
#Depression	[1,10]
#Insomnia	[1,10]
#Hours per day	[0,24]

#Parametros
edad = pnw.IntInput(name='Edad', start=10, end=89,step=1, width=200)

ansiedad = pnw.FloatSlider(name='Ansiedad', start=1, end=10.0, step=0.5, width=200)
depresion = pnw.FloatSlider(name='Depresión', start=1, end=10.0, step=0.5, width=200)
insomnio = pnw.FloatSlider(name='Insomnio', start=1, end=10.0, step=0.5, width=200)
horas_dia = pnw.FloatSlider(name='Horas por día', start=1, end=10.0, step=0.5, width=200)

#CATEGORICAS
#While working	[Si, No]
#Instrumentalist	[Si, No]
#Composer	[Si, No]
#Exploratory	[Si, No]
#Foreign languages [Si, No]
#Primary streaming service [Apple Music, Sin servicio, Otro, Pandora, Spotify, Youtube Music]
#Fav genre	[Clasica, Country, EDM, Folk, Gospel, Hip-Hop, Jazz, K-pop, Latin, Lo-Fi, Metal, Pop, R&B, Rap, Rock, Videjuegos]
#Music effects	(Mejoro,Empeoro,Sin cambios)
#(Nunca, Rara vez, Algunas veces, Con frecuencia)
#Frequency [Classical]	Frequency [Country]	Frequency [EDM]	Frequency [Folk]	Frequency [Gospel]	Frequency [Hip hop]	Frequency [Jazz]
#Frequency [K pop]	Frequency [Latin]	Frequency [Lofi]	Frequency [Metal]	Frequency [Pop]	Frequency [R&B]	Frequency [Rap]	Frequency [Rock]
#Frequency [Video game music]

trabajando = pnw.Select(name='Trabajando', options=['Sí', 'No']	, width=200)
instrumental = pnw.Select(name='Instrumental', options=['Sí', 'No']	, width=200)
compositor = pnw.Select(name='Compositor', options=['Sí', 'No']	, width=200)
exploracion = pnw.Select(name='Exploración', options=['Sí', 'No']	, width=200)
extranjero = pnw.Select(name='Extranjero', options=['Sí', 'No']	, width=200)
servicio_streaming = pnw.Select(name='Servicio Striming', options=['Apple Music', 'Sin servicio', 'Otro', 'Pandora', 'Spotify', 'Youtube Music']	, width=200)
genero = pnw.Select(name='Género', options=['Clasica', 'Country', 'EDM', 'Folk', 'Gospel', 'Hip-Hop', 'Jazz', 'K-pop', 'Latin', 'Lo-Fi', 'Metal',
                                            'Pop', 'R&B', 'Rap', 'Rock', 'Videjuegos']	, width=200)
frecuencia = pnw.Select(name='Frecuencia', options=['Nunca', 'Rara vez', 'Algunas veces', 'Con frecuencia'])

# Graficas
bar_plot = pn.pane.Plotly()
hist_plot = pn.pane.Plotly()
disp_plot = pn.pane.Plotly()

dynamic_widgets_1 = pn.Row()
dynamic_widgets_2 = pn.Row()

def update_widgets_1(datasets):
  if datasets == 'Origen':
    return []
  else:
    return [edad,ansiedad,depresion,insomnio,horas_dia,trabajando]

def update_widgets_2(datasets):
  if datasets == 'Origen':
    return []
  else:
    return [instrumental,compositor,exploracion,extranjero,servicio_streaming,genero,frecuencia]

def update_plots():
  edad = edad.value,ansiedad.value,depresion.value,insomnio.value,horas_dia.value,trabajando.value,instrumental.value,compositor.value,exploracion.value,extranjero.value,servicio_streaming.value,genero.value,frecuencia.value
  return

In [8]:
#Dependencias

#Cambio de trayectoria
@pn.depends(datasets.param.value)
def parameters_1(datasets):
  dynamic_widgets_1.clear()
  widgets = update_widgets_1(datasets)
  if widgets:
      dynamic_widgets_1.extend(widgets)
  return dynamic_widgets_1

@pn.depends(datasets.param.value)
def parameters_2(datasets):
  dynamic_widgets_2.clear()
  widgets = update_widgets_2(datasets)
  if widgets:
      dynamic_widgets_2.extend(widgets)
  return dynamic_widgets_2

#Cambio de parametros
@pn.depends(datasets.param.value,edad.param.value,ansiedad.param.value,depresion.param.value,insomnio.param.value,horas_dia.param.value,trabajando.param.value,instrumental.param.value,compositor.param.value,exploracion.param.value,extranjero.param.value,servicio_streaming.param.value,genero.param.value,frecuencia.param.value)
def check_all_widgets(*args):
  update_plots()

In [9]:
#Grafica de barras (Totales)
bar_options = pnw.Select(name='Columns', options=['Age', 'Primary streaming service', 'Hours per day', 'Fav genre'], width=200)

@pn.depends(bar_options.param.value)
def update_bar_plot(*args):
  global bar_plot
  bar_plot = generate_bar_plot(x=dataset[bar_options.value], title=bar_options.value)
  return bar_plot

disp_options_x = pnw.Select(name='Column X', options=['Age', 'Primary streaming service', 'Hours per day', 'Fav genre'], width=200)
disp_options_y = pnw.Select(name='Column Y', options=['Age', 'Primary streaming service', 'Hours per day', 'Fav genre'], width=200)

@pn.depends(disp_options_x.param.value,disp_options_y.param.value)
def update_disp_plot(*args):
  disp_plot = generate_disp_plot(x=dataset[disp_options_x.value], y=dataset[disp_options_y.value], title=f"{disp_options_x.value} vs {disp_options_y.value}")
  return disp_plot

In [10]:
pn.config.sizing_mode = 'stretch_width'  # Para limitar la expansión horizontal

# Crea tu dashboard aquí
dashboard = pn.Row(
    #pn.Row(datasets),
    #parameters_1,
    #parameters_2,
    #pn.Row(hist_plot),
    pn.Column(bar_options, update_bar_plot),
    pn.Column(pn.Row(disp_options_x, disp_options_y), update_disp_plot)
)

pn.config.max_width = 300  # Establece el ancho máximo
pn.config.max_height = 300  # Establece la altura máxima

dashboard.servable(title='Dashboard')

Row(sizing_mode='stretch_width')
    [0] Column(sizing_mode='stretch_width')
        [0] Select(name='Columns', options=['Age', 'Primary streaming...], value='Age', width=200)
        [1] ParamFunction(function, _pane=Plotly, defer_load=False, sizing_mode='stretch_width')
    [1] Column(sizing_mode='stretch_width')
        [0] Row(sizing_mode='stretch_width')
            [0] Select(name='Column X', options=['Age', 'Primary streaming...], value='Age', width=200)
            [1] Select(name='Column Y', options=['Age', 'Primary streaming...], value='Age', width=200)
        [1] ParamFunction(function, _pane=Plotly, defer_load=False, sizing_mode='stretch_width')